In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import pandas as pd
import firebase_admin
from firebase_admin import credentials, firestore, db
import json

In [2]:
# firebase login stuff

### make a credential json...

cred = credentials.Certificate("C:\\Users\\erussek\\forage_jsp\\analysis\\foragetask-firebase-private-key.json")
default_app = firebase_admin.initialize_app(cred)
client = firestore.client()

In [22]:
dfs = []
bonus_df = pd.DataFrame()
Qdfs = []
for subj in client.collection('foragetask').document('run4').collection('subjects').stream():
    subj_dfs = []
    taskdata_collection = client.collection('foragetask/run4/subjects/{0}/taskdata'.format(subj.id)).stream()
    has_end = False # did this subject finish the task?
   # has_start = False
    subjectID = subj.id
    for tc in taskdata_collection:
        # first thing is start... 
        if tc.id == "start":
            has_start = True
            start_dict = tc.to_dict()
            subjectID = start_dict['subjectID']  
        elif tc.id == "end":
            print(subj.id)
            has_end = True
            end_dict = tc.to_dict()
            end_time = end_dict['end_time']
            bonus_points = end_dict['bonus_points'][0]
        elif tc.id == "Q_data":
            has_start = True
            Q_dict = tc.to_dict()
            Q_data_json = json.loads(Q_dict['trial_data'])
            Q_df = pd.DataFrame(Q_data_json)
            Q_df['subjectID'] = subj.id
            Qdfs.append(Q_df)

            #subjectID = start_dict['subjectID']
            #date = start_dict['date']
        else:
            trial_dict = tc.to_dict()
            trial_name = tc.id
            trial_data_json = json.loads(trial_dict['trial_data'])
            trial_df = pd.DataFrame(trial_data_json)
            trial_df['subjectID'] = subjectID
            #if has_start:
            #    trial_df['subjectID'] = subjectID
            #    trial_df['date'] = date
            #else:
            #    trial_df['subjectID'] = None
            #    trial_df['date'] = None
            if has_end:
                trial_df['bonus_points'] = bonus_points
            else:
                trial_df['bonus_points'] = None
                
            subj_dfs.append(trial_df)
            
    if (has_end & (len(str(subjectID)) > 20)):
        bonus_dict = {'subjectID': subjectID, 'bonus_points': bonus_points}
        bonus_df = bonus_df.append(bonus_dict, ignore_index = True)
        dfs = dfs + subj_dfs


combined_df = pd.concat(dfs, sort = True)
combined_df.to_csv('data\\run4_data.csv')

# make a structure w/ bonus points... 
def compute_bonus(row):
    pct = np.min([np.max([row['bonus_points'] / (30000 - 0),0]),1])
    return np.around(2.00*pct, decimals = 2)
                  
bonus_df['bonus'] = bonus_df.apply(compute_bonus, axis = 1)
bonus_df[['subjectID', 'bonus']].to_csv('bonus_payment_files\\run1_bonus_payments.csv', sep = ',', header = False, index = False)


0RPIlTRm15S9OoQUTRMMVLqB4mB3
2DBQvX2mbferLX20F34ihCi5G9O2
4TXib0e9FSeU3YbBqCEhxHcIRPo2
5ybe3Oy53VgsWBxBww3sLaNPcjP2
60CDvwB7TnVXIhjjFf4cWUiiZpx1
CtnOnkTcZUUiDSHhd5dCIiM93v93
KhgXbNhgJBMTn13irityCpLskso1
LJiWhWqbK0dt0sMvzYiBr4Ge6tz2
QDLvEgFkuXN4UjXerkR8AV1iO9d2
RIAEb7ltPRY1BUjMziUfzgvbqr32
TJP4y5O4htQ5eZAtjTCl2Krg2kn2
WMuRxvkg9kerZvf4OA30FQignUQ2
WfIGLqwm3vT1jEMCqu9X0ohNVNF3
XBKbOP2CQNbHZCRrWGDjjOJsImE3
YDgYXJV05PUXInfhh9FHAgHrgzi2
blZ3YzimTGbmALWJGvKHk27HOMa2
e58mQ00ZSnRyOy2LdWeo7EReair2
jqx41BdenXP2rOfLb68KWVjv0Jt2
kWf6p8syLoXpJKkiFH7MneBKkCJ2
ncwtZP4zH9OXrZ6vqXltXhDsf543
uZKtvEH3VAdnzL8YznObRD9473d2
wGbx18iemCTNIPAMUxYbXUyZYXG2
wpZ3nqzaHGh26Sah8LHIjjdbcQv1


TypeError: "delimiter" must be a 1-character string

In [23]:
bonus_df[['subjectID', 'bonus']].to_csv('bonus_payment_files\\run1_bonus_payments.csv', sep = ',', header = False, index = False)

In [7]:
end_dict

{'bonus_points': [], 'end_time': '11:40:00 AM'}

In [10]:
combined_df

bonus_points correct_key  decay  exit  fail  \
0           16121        True   0.98   NaN   NaN   
1           16121        True   0.98   NaN   NaN   
2           16121        True   0.98   NaN   NaN   
3           16121        True   0.98   NaN   NaN   
4           16121        True   0.98   NaN   NaN   
..            ...         ...    ...   ...   ...   
763         16121        True   0.98   0.0   NaN   
764         16121        True   0.98   0.0   NaN   
765         16121        True   0.98   0.0   NaN   
766         16121        True   0.98   0.0   NaN   
767         16121        True   0.98   0.0   NaN   

                                           harvest_key internal_node_id key  \
0    [Repeatedly press 'u' to harvest or 'f' to tra...         0.0-11.0   f   
1    [Repeatedly press 'u' to harvest or 'f' to tra...         0.0-11.0   f   
2    [Repeatedly press 'u' to harvest or 'f' to tra...         0.0-11.0   f   
3    [Repeatedly press 'u' to harvest or 'f' to tra...         0.0-11.0   f   
4    [Repeatedly press 'u' to harvest or 'f' to tra...         0.0-11.0   f   
..                                                 ...              ...  ..   
763  [Repeatedly press 'u' to harvest or 'f' to tra...         0.0-21.0   u   
764  [Repeatedly press 'u' to harvest or 'f' to tra...         0.0-21.0   u   
765  [Repeatedly press 'u' to harvest or 'f' to tra...         0.0-21.0   u   
766  [Repeatedly press 'u' to harvest or 'f' to tra...         0.0-21.0   u   
767  [Repeatedly press 'u' to harvest or 'f' to tra...         0.0-21.0   u   

    keys_held_down       lag  ...                     subjectID  success  \
0              NaN  1366.000  ...  4Lq2oS0bWlXhLIXjDo2AD8gFcnn2     True   
1              NaN   186.000  ...  4Lq2oS0bWlXhLIXjDo2AD8gFcnn2     True   
2              NaN   165.000  ...  4Lq2oS0bWlXhLIXjDo2AD8gFcnn2     True   
3              NaN   178.000  ...  4Lq2oS0bWlXhLIXjDo2AD8gFcnn2    False   
4              NaN   143.000  ...  4Lq2oS0bWlXhLIXjDo2AD8gFcnn2     True   
..             ...       ...  ...                           ...      ...   
763            NaN   141.820  ...  vNd3QJdQBAfnBa0WZcn84rBCfRc2    False   
764            NaN   133.560  ...  vNd3QJdQBAfnBa0WZcn84rBCfRc2     True   
765            NaN   182.175  ...  vNd3QJdQBAfnBa0WZcn84rBCfRc2     True   
766            NaN   158.735  ...  vNd3QJdQBAfnBa0WZcn84rBCfRc2    False   
767            NaN   143.105  ...  vNd3QJdQBAfnBa0WZcn84rBCfRc2     True   

    time_elapsed  time_min  total_points  \
0         419050      2.25             0   
1         419236      2.25             0   
2         419401      2.25             0   
3         419579      2.25             0   
4         419722      2.25             0   
..           ...       ...           ...   
763      2213769      2.25         20999   
764      2213907      2.25         21080   
765      2214091      2.25         21153   
766      2214251      2.25         21153   
767      2214396      2.25         21232   

                                            travel_key  tree_pos  trial_index  \
0    [Repeatedly press 'f' (index) while holding do...        13            9   
1    [Repeatedly press 'f' (index) while holding do...        13            9   
2    [Repeatedly press 'f' (index) while holding do...        13            9   
3    [Repeatedly press 'f' (index) while holding do...        13            9   
4    [Repeatedly press 'f' (index) while holding do...        13            9   
..                                                 ...       ...          ...   
763  [Repeatedly press 'f' (index) while holding do...        68           19   
764  [Repeatedly press 'f' (index) while holding do...        68           19   
765  [Repeatedly press 'f' (index) while holding do...        68           19   
766  [Repeatedly press 'f' (index) while holding do...        68           19   
767  [Repeatedly press 'f' (index) while holding do...        68           19   

     trial_num  